# Étape 3 — Indicateurs Clés de Performance (ICP)

L’objectif de cette étape est de calculer et analyser les principaux indicateurs
de performance commerciale et de satisfaction client à partir des données
nettoyées issues de l’étape précédente.

Les indicateurs sont construits en privilégiant des mesures économiquement
pertinentes, notamment des taux calculés en valeur lorsque cela est justifié.


**Import des librairies**

In [1]:
# Import des librairies nécessaires à l'analyse des ICP

import pandas as pd
import numpy as np

# Options d'affichage pour une meilleure lisibilité
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

**Chargement des données nettoyées**

In [2]:
# Chargement des données nettoyées depuis le dossier data/processed

order_lines = pd.read_csv("../data/processed/order_lines_clean.csv")
customers = pd.read_csv("../data/processed/customers_clean.csv")
products = pd.read_csv("../data/processed/products_clean.csv")

**Préparation des données pour l’analyse ICP**

In [3]:
# Conversion des variables de date
order_lines["order_date"] = pd.to_datetime(order_lines["order_date"])

# Création de la variable mois de commande
order_lines["order_month"] = order_lines["order_date"].dt.to_period("M")

# Vérification rapide de la structure des données
order_lines[[
    "order_id",
    "order_date",
    "order_month",
    "net_amount",
    "quantity",
    "returned",
    "category",
    "channel",
    "review_score",
    "delivery_days"
]].head()


,order_id,order_date,order_month,net_amount,quantity,returned,category,channel,review_score,delivery_days
0,O00001,2024-06-16,2024-06,542.97,4,0,Cours,App,5.00,1.00
1,O00002,2024-06-10,2024-06,199.86,5,0,Livre,Web,4.00,0.00
2,O00003,2024-08-29,2024-08,"2,157.10",2,0,Laptop,Web,4.00,3.00
3,O00004,2024-01-27,2024-01,106.56,2,0,Cloud,Boutique,4.00,0.00
4,O00005,2025-08-06,2025-08,"4,812.00",3,0,Laptop,Web,5.00,9.00


**Chiffre d’affaires total (CA)**

In [4]:
# Calcul du chiffre d'affaires total (CA)
# Le CA est calculé à partir du montant net (net_amount),
# c’est-à-dire après application des remises.

ca_total = order_lines["net_amount"].sum()

# Affichage du résultat
print(f"Chiffre d'affaires total (CA net) : {ca_total:,.2f}")

Chiffre d'affaires total (CA net) : 2,023,658.44


**Chiffre d’affaires mensuel**

In [5]:
# Calcul du chiffre d'affaires mensuel à partir du CA net
# Agrégation par mois de commande (order_month)

ca_mensuel = (
    order_lines
    .groupby("order_month", as_index=False)
    .agg(CA_mensuel=("net_amount", "sum"))
    .sort_values("order_month")
)

# Affichage du chiffre d'affaires mensuel
ca_mensuel


,order_month,CA_mensuel
0,2024-01,"96,136.26"
1,2024-02,"124,550.19"
2,2024-03,"85,860.28"
3,2024-04,"82,740.55"
4,2024-05,"73,921.77"
5,2024-06,"95,653.19"
6,2024-07,"85,929.85"
7,2024-08,"108,552.22"
8,2024-09,"72,245.42"
9,2024-10,"67,970.70"


**Export du chiffre d'affaires mesnsuel vers results/tables**

In [6]:
ca_mensuel.to_csv("../results/tables/ca_mensuel.csv", index=False)

**chiffre d'affaires net par catégorie et par canal**

In [7]:
# Tableau croisé : chiffre d'affaires net par catégorie et par canal
# Utilisation explicite de pivot_table (analyse multidimensionnelle)

ca_pivot = pd.pivot_table(
    order_lines,
    values="net_amount",
    index="category",
    columns="channel",
    aggfunc="sum",
    margins=True
)

ca_pivot

channel,App,Boutique,Web,All
category,,,,
Accessoire,"16,266.46","5,422.43","28,837.82","50,526.71"
Cloud,"43,484.46","17,074.84","94,413.85","154,973.15"
Cours,"77,491.25","19,463.38","106,234.58","203,189.21"
Laptop,"483,951.84","216,430.69","710,324.11","1,410,706.64"
Livre,"13,672.97","4,214.30","22,815.19","40,702.46"
Logiciel,"54,162.40","20,319.67","89,078.20","163,560.27"
All,"689,029.38","282,925.31","1,051,703.75","2,023,658.44"


**Export du chiffre d'affaires net par catégorie et par canal vers results/tables**

In [8]:
ca_pivot.to_csv("../results/tables/ca_pivot.csv", index=False)

**Panier moyen global (AOV)**

In [9]:
# Calcul du panier moyen global (Average Order Value - AOV)
# AOV = Chiffre d'affaires total / Nombre de commandes distinctes

# Nombre total de commandes
nb_commandes = order_lines["order_id"].nunique()

# Panier moyen global
aov_global = ca_total / nb_commandes

# Affichage du résultat
print(f"Panier moyen global (AOV) : {aov_global:,.2f}")


Panier moyen global (AOV) : 924.89


**Panier moyen mensuel (AOV mensuel)**

In [10]:
# Calcul du panier moyen mensuel (AOV mensuel)
# AOV mensuel = CA mensuel / Nombre de commandes mensuelles

aov_mensuel = (
    order_lines
    .groupby("order_month")
    .agg(
        CA_mensuel=("net_amount", "sum"),
        nb_commandes=("order_id", "nunique")
    )
    .assign(AOV_mensuel=lambda df: df["CA_mensuel"] / df["nb_commandes"])
    .reset_index()
    .sort_values("order_month")
)

# Affichage du panier moyen mensuel
aov_mensuel

,order_month,CA_mensuel,nb_commandes,AOV_mensuel
0,2024-01,"96,136.26",100,961.36
1,2024-02,"124,550.19",88,"1,415.34"
2,2024-03,"85,860.28",90,954.00
3,2024-04,"82,740.55",101,819.21
4,2024-05,"73,921.77",96,770.02
5,2024-06,"95,653.19",96,996.39
6,2024-07,"85,929.85",99,867.98
7,2024-08,"108,552.22",111,977.95
8,2024-09,"72,245.42",75,963.27
9,2024-10,"67,970.70",79,860.39


**Export du panier moyen mesnsuel vers results/tables**

In [11]:
aov_mensuel.to_csv("../results/tables/aov_mensuel.csv" , index=False)

## Méthodologie de calcul du taux de remise

Les taux de remise peuvent être calculés soit en volume (nombre d’articles),
soit en valeur monétaire. Dans le cadre de ce projet, le calcul est effectué
**en valeur**, afin de tenir compte de l’hétérogénéité des prix entre les
produits et les catégories.

Le calcul en volume attribuerait le même poids à des articles de faible valeur
et à des articles de valeur élevée, ce qui pourrait biaiser l’analyse
économique. À l’inverse, le calcul en valeur permet de mesurer l’impact réel
des remises sur le chiffre d’affaires.

**Taux de remise moyen global (en valeur)**

In [12]:
# Calcul du taux de remise moyen global (en valeur)

# Calcul du prix catalogue (avant remise)
order_lines["catalog_amount"] = order_lines["unit_price"] * order_lines["quantity"]

# Calcul du montant total des remises
total_discount_value = (order_lines["catalog_amount"] - order_lines["net_amount"]).sum()

# Calcul du montant total catalogue
total_catalog_value = order_lines["catalog_amount"].sum()

# Taux de remise moyen global (en valeur)
taux_remise_global = total_discount_value / total_catalog_value

# Affichage du résultat
print(f"Taux de remise moyen global (en valeur) : {taux_remise_global:.2%}")

Taux de remise moyen global (en valeur) : 11.66%


**Taux de remise moyen par catégorie (en valeur)**

In [13]:
# Calcul du taux de remise moyen par catégorie (en valeur)

taux_remise_par_categorie = (
    order_lines
    .groupby("category")
    .agg(
        discount_value=("catalog_amount", 
                        lambda x: (x - order_lines.loc[x.index, "net_amount"]).sum()),
        catalog_value=("catalog_amount", "sum")
    )
    .assign(taux_remise=lambda df: df["discount_value"] / df["catalog_value"])
    .sort_values("taux_remise", ascending=False)
    .reset_index()
)

# Affichage du taux de remise par catégorie
taux_remise_par_categorie

,category,discount_value,catalog_value,taux_remise
0,Laptop,"189,050.01","1,599,756.65",0.12
1,Livre,"5,380.14","46,082.60",0.12
2,Cours,"26,515.40","229,704.61",0.12
3,Accessoire,"6,520.36","57,047.07",0.11
4,Cloud,"19,753.85","174,727.00",0.11
5,Logiciel,"19,904.92","183,465.19",0.11


**Export du taux de remise par catégorie vers results/tables**

In [14]:
taux_remise_par_categorie.to_csv("../results/tables/taux_remise_par_categorie.csv", index=False)

**Top 5 catégories les plus remisées**

In [15]:
# Top 5 catégories avec le taux de remise le plus élevé

top_5_remises = taux_remise_par_categorie.nlargest(5, "taux_remise")

top_5_remises

,category,discount_value,catalog_value,taux_remise
0,Laptop,"189,050.01","1,599,756.65",0.12
1,Livre,"5,380.14","46,082.60",0.12
2,Cours,"26,515.40","229,704.61",0.12
3,Accessoire,"6,520.36","57,047.07",0.11
4,Cloud,"19,753.85","174,727.00",0.11


**Export du top_5 vers results/tables**

In [16]:
top_5_remises.to_csv("../results/tables/top_5_remises.csv", index=False)

## Méthodologie de calcul des taux de retour

Les taux de retour peuvent être calculés soit en volume (nombre d’articles ou de
commandes retournées), soit en valeur monétaire. Dans le cadre de ce projet,
l’analyse privilégie les **taux de retour calculés en valeur**, considérés comme
les plus pertinents d’un point de vue économique.

En effet, le calcul en volume attribue le même poids à toutes les unités
vendues, indépendamment de leur prix. Or, le jeu de données étudié présente une
forte hétérogénéité des montants de vente selon les produits, les catégories et
les canaux de distribution.

Le calcul en valeur permet de mesurer l’impact réel des retours sur le chiffre
d’affaires, en rapportant le montant des ventes retournées au montant total des
ventes. Cette approche est particulièrement adaptée pour l’aide à la décision
économique et commerciale.

**Taux de retour global**

In [17]:
# Calcul du taux de retour global

# --- Taux de retour en volume ---
# Nombre total d'articles vendus
articles_vendus = order_lines["quantity"].sum()

# Nombre total d'articles retournés
articles_retournes = order_lines.loc[order_lines["returned"] == 1, "quantity"].sum()

taux_retour_volume = articles_retournes / articles_vendus


# --- Taux de retour en valeur ---
# Montant total des ventes
valeur_ventes_totales = order_lines["net_amount"].sum()

# Montant des ventes retournées
valeur_ventes_retournees = order_lines.loc[
    order_lines["returned"] == 1, "net_amount"
].sum()

taux_retour_valeur = valeur_ventes_retournees / valeur_ventes_totales


# Affichage des résultats
print(f"Taux de retour global (en volume) : {taux_retour_volume:.2%}")
print(f"Taux de retour global (en valeur) : {taux_retour_valeur:.2%}")

Taux de retour global (en volume) : 4.30%
Taux de retour global (en valeur) : 3.77%


**Taux de retour par catégorie (en valeur)**

In [18]:
# Calcul du taux de retour par catégorie (en valeur)

taux_retour_par_categorie = (
    order_lines
    .groupby("category")
    .agg(
        ventes_totales=("net_amount", "sum"),
        ventes_retournees=("net_amount", 
                            lambda x: x[order_lines.loc[x.index, "returned"] == 1].sum())
    )
    .assign(taux_retour=lambda df: df["ventes_retournees"] / df["ventes_totales"])
    .sort_values("taux_retour", ascending=False)
    .reset_index()
)

# Affichage du taux de retour par catégorie
taux_retour_par_categorie

,category,ventes_totales,ventes_retournees,taux_retour
0,Accessoire,"50,526.71","3,142.01",0.06
1,Livre,"40,702.46","1,859.28",0.05
2,Cours,"203,189.21","9,055.28",0.04
3,Laptop,"1,410,706.64","52,226.64",0.04
4,Logiciel,"163,560.27","5,635.58",0.03
5,Cloud,"154,973.15","4,323.38",0.03


**Export du taux de retour par catégorie vers results/tables**

In [19]:
taux_retour_par_categorie.to_csv("../results/tables/taux_retour_par_categorie.csv", index=False)

**Taux de retour par canal (en valeur)**

In [20]:
# Calcul du taux de retour par canal de vente (en valeur)

taux_retour_par_canal = (
    order_lines
    .groupby("channel")
    .agg(
        ventes_totales=("net_amount", "sum"),
        ventes_retournees=("net_amount", 
                            lambda x: x[order_lines.loc[x.index, "returned"] == 1].sum())
    )
    .assign(taux_retour=lambda df: df["ventes_retournees"] / df["ventes_totales"])
    .sort_values("taux_retour", ascending=False)
    .reset_index()
)

# Affichage du taux de retour par canal
taux_retour_par_canal

,channel,ventes_totales,ventes_retournees,taux_retour
0,Web,"1,051,703.75","49,742.87",0.05
1,Boutique,"282,925.31","8,003.60",0.03
2,App,"689,029.38","18,495.70",0.03


**Export du taux de retour par canal vers results/tables**

In [21]:
taux_retour_par_canal.to_csv("../results/tables/taux_retour_par_canal.csv", index=False)

**Top 5 catégories avec le plus fort taux de retour (en valeur)**

In [22]:
# Top 5 catégories avec le taux de retour le plus élevé

top_5_retours = taux_retour_par_categorie.nlargest(5, "taux_retour")

top_5_retours


,category,ventes_totales,ventes_retournees,taux_retour
0,Accessoire,"50,526.71","3,142.01",0.06
1,Livre,"40,702.46","1,859.28",0.05
2,Cours,"203,189.21","9,055.28",0.04
3,Laptop,"1,410,706.64","52,226.64",0.04
4,Logiciel,"163,560.27","5,635.58",0.03


**Export du taux de retours vers results/tables**

In [23]:
top_5_retours.to_csv("../results/tables/top_5_retours.csv", index=False)

**Score d’avis moyen global**

In [24]:
# Calcul du score d'avis moyen global
# Moyenne simple des scores d'avis après nettoyage

score_avis_global = order_lines["review_score"].mean()

# Affichage du résultat
print(f"Score d'avis moyen global : {score_avis_global:.2f}")

Score d'avis moyen global : 4.36


**Score d’avis moyen par catégorie**

In [25]:
# Calcul du score d'avis moyen par catégorie de produit

score_avis_par_categorie = (
    order_lines
    .groupby("category")
    .agg(score_moyen=("review_score", "mean"))
    .sort_values("score_moyen", ascending=False)
    .reset_index()
)

# Affichage du score d'avis moyen par catégorie
score_avis_par_categorie

,category,score_moyen
0,Accessoire,4.42
1,Cours,4.38
2,Logiciel,4.36
3,Laptop,4.33
4,Livre,4.33
5,Cloud,4.30


**Export du score moyen par catégorie vers results/tables**

In [26]:
score_avis_par_categorie.to_csv("../results/tables/score_avis_par_categorie.csv", index=False)

**Création des intervalles de délai de livraison**

In [27]:
# Création d'intervalles numériques pour le délai de livraison (en jours)
# Intervalles retenus :
# 0–2 jours : livraison très rapide
# 3–5 jours : livraison standard
# 6–10 jours : livraison lente
# > 10 jours : livraison très tardive

bins = [0, 2, 5, 10, np.inf]
labels = ["0-2 jours", "3-5 jours", "6-10 jours", "> 10 jours"]

order_lines["delivery_delay_class"] = pd.cut(
    order_lines["delivery_days"],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

# Vérification rapide de la répartition par classe de délai
order_lines["delivery_delay_class"].value_counts().sort_index()

delivery_delay_class
0-2 jours     1003
3-5 jours      637
6-10 jours     301
> 10 jours     247
Name: count, dtype: int64

**Score d’avis moyen par délai de livraison**

In [28]:
# Calcul du score d'avis moyen par intervalle de délai de livraison

score_avis_par_delai = (
    order_lines
    .groupby("delivery_delay_class")
    .agg(
        score_moyen=("review_score", "mean"),
        nb_commandes=("order_id", "nunique")
    )
    .reset_index()
)

# Affichage des résultats
score_avis_par_delai

C:\Users\DELL\AppData\Local\Temp\ipykernel_8820\590399187.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("delivery_delay_class")


,delivery_delay_class,score_moyen,nb_commandes
0,0-2 jours,4.34,1003
1,3-5 jours,4.39,637
2,6-10 jours,4.34,301
3,> 10 jours,4.33,247


**Export score_avis par delai vers results/tables**

In [29]:
score_avis_par_delai.to_csv("../results/tables/score_avis_par_delai.csv", index=False)